In [1]:
#!pip3 install ray torch torchvision tabulate tensorboard
#!pip3 install 'ray[rllib]'
#!pip3 install ray

In [1]:
import torch 
from torch import nn

import ray
from ray.rllib.agents import ppo
from ray.rllib.models import ModelCatalog
from ray.rllib.models.torch.torch_modelv2 import TorchModelV2
from ray.rllib.utils.annotations import override

from models import VisualEncoder
from train import *

/root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [2]:
class MyModelClass(TorchModelV2, nn.Module):
    def __init__(self, obs_space, action_space, num_outputs, model_config, name):
        TorchModelV2.__init__(self, obs_space, action_space, num_outputs, model_config, name)
        nn.Module.__init__(self)
        features_dim = 512
        self.encoder = VisualEncoder(features_dim)
        self.encoder.load_state_dict(
            torch.load("/src/Visual Autoencoder weights and models/IGLU_encoder_weigths.pth", map_location=torch.device('cpu'))
        )
        self.action_head = nn.Linear(features_dim, action_space.n)
        self.value_head = nn.Linear(features_dim, 1)
        self.last_value = None
        
        self.use_cuda = torch.cuda.is_available()
        if self.use_cuda:
            self.encoder.cuda()
            self.action_head.cuda()
            self.value_head.cuda()
        
    @override(TorchModelV2)
    def forward(self, input_dict, state, seq_lens):
        obs = input_dict['obs'].permute(0, 3, 1, 2).float() / 255.0
        if self.use_cuda:
            obs.cuda()
            
        features = self.encoder(obs)
        action = self.action_head(features)
        self.last_value = self.value_head(features).squeeze(1)
        return action, state
    
    @override(TorchModelV2)
    def value_function(self):
        assert self.last_value is not None, "must call forward() first"
        return self.last_value

In [3]:
ModelCatalog.register_custom_model("my_torch_model", MyModelClass)

In [4]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

def env_creator(env_config):
    env = gym.make('IGLUSilentBuilder-v0', max_steps=1000)
    env.update_taskset(TaskSet(preset=['C17']))
    env = PovOnlyWrapper(env)
    env = IgluActionWrapper(env)
    return env

from ray.tune.registry import register_env
register_env("my_env", env_creator)

from ray import tune
from ray.rllib.agents.ppo import PPOTrainer

In [ ]:
from ray.tune.integration.wandb import WandbLogger

tune.run(PPOTrainer, 
         config={
             "env": "my_env", 
             "framework": "torch",
             "num_gpus": 1,
             "num_workers": 1,
             "sgd_minibatch_size": 256,
             "clip_param": 0.2,
             "entropy_coeff": 0.01,
             "lambda": 0.95,
             "train_batch_size": 1000,
             "model": {
                    # Specify our custom model from above.
                    "custom_model": "my_torch_model",
                    # Extra kwargs to be passed to your model's c'tor.
                    "custom_model_config": {},
              },
             "logger_config": {
                  "wandb": {
                      "project": "IGLU-Minecraft",
                      "name": "PPO C17 pretrained"
                  }
              }

        },
        loggers=[WandbLogger])#callbacks=[
        #    CustomLoggerCallback(),
        #])

/root/miniconda/envs/py37/lib/python3.7/site-packages/ray/_private/services.py:238: UserWarning: Not all Ray Dashboard dependencies were found. To use the dashboard please install Ray using `pip install ray[default]`. To disable this message, set RAY_DISABLE_IMPORT_WARNING env var to '1'.
  warnings.warn(warning_message)


Trial name,status,loc
PPO_my_env_d74d3_00000,PENDING,


2021-09-17 11:31:46,757	INFO wandb.py:170 -- Already logged into W&B.
2021-09-17 11:31:46,811	ERROR syncer.py:72 -- Log sync requires rsync to be installed.
wandb: Currently logged in as: linar (use `wandb login --relogin` to force relogin)
(pid=333) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=333)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=333) 2021-09-17 11:31:51,791	INFO ppo.py:159 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
(pid=333) 2021-09-17 11:31:51,791	INFO trainer.py:728 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


(pid=334) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=334)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=333) 2021-09-17 11:32:04,937	INFO trainable.py:109 -- Trainable.setup took 16.441 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
(pid=333) 2021-09-17 11:32:04,939	WARNING util.py:55 -- Install gputil for GPU system monitoring.
(pid=334) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=334)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=334) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=334)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=334) 

Result for PPO_my_env_d74d3_00000:
  agent_timesteps_total: 1000
  custom_metrics: {}
  date: 2021-09-17_11-37-32
  done: false
  episode_len_mean: 1000.0
  episode_media: {}
  episode_reward_max: 0.0
  episode_reward_mean: 0.0
  episode_reward_min: 0.0
  episodes_this_iter: 1
  episodes_total: 1
  experiment_id: 07f70cbd5fb847dfafeefd9a680181c0
  hostname: cds2
  info:
    learner:
      default_policy:
        learner_stats:
          cur_kl_coeff: 0.19999999999999996
          cur_lr: 5.000000000000001e-05
          entropy: 1.8325520820087857
          entropy_coeff: 0.009999999999999998
          kl: 0.022896651729185865
          policy_loss: 0.14715835692154036
          total_loss: 0.1939159910298056
          vf_explained_var: 0.15425756573677063
          vf_loss: 0.06050382106461459
    num_agent_steps_sampled: 1000
    num_agent_steps_trained: 1000
    num_steps_sampled: 1000
    num_steps_trained: 1000
  iterations_since_restore: 1
  node_ip: 10.55.229.87
  num_healthy_wor

...

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_my_env_d74d3_00000,RUNNING,10.55.229.87:333,641,13876.8,641000,0,0,0,995.57


Result for PPO_my_env_d74d3_00000:
  agent_timesteps_total: 642000
  custom_metrics: {}
  date: 2021-09-17_15-24-05
  done: false
  episode_len_mean: 995.57
  episode_media: {}
  episode_reward_max: 0.0
  episode_reward_mean: 0.0
  episode_reward_min: 0.0
  episodes_this_iter: 1
  episodes_total: 644
  experiment_id: 07f70cbd5fb847dfafeefd9a680181c0
  hostname: cds2
  info:
    learner:
      default_policy:
        learner_stats:
          cur_kl_coeff: 0.26362734606122995
          cur_lr: 5.000000000000001e-05
          entropy: 2.652866564856635
          entropy_coeff: 0.009999999999999998
          kl: 0.005862735403117369
          policy_loss: -0.05096826437446806
          total_loss: -0.07583956658426258
          vf_explained_var: -0.8788576126098633
          vf_loss: 0.0001117869686267012
    num_agent_steps_sampled: 642000
    num_agent_steps_trained: 642000
    num_steps_sampled: 642000
    num_steps_trained: 642000
  iterations_since_restore: 642
  node_ip: 10.55.229.87

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_my_env_d74d3_00000,RUNNING,10.55.229.87:333,642,13895.4,642000,0,0,0,995.57


Result for PPO_my_env_d74d3_00000:
  agent_timesteps_total: 643000
  custom_metrics: {}
  date: 2021-09-17_15-24-25
  done: false
  episode_len_mean: 995.57
  episode_media: {}
  episode_reward_max: 0.0
  episode_reward_mean: 0.0
  episode_reward_min: 0.0
  episodes_this_iter: 1
  episodes_total: 645
  experiment_id: 07f70cbd5fb847dfafeefd9a680181c0
  hostname: cds2
  info:
    learner:
      default_policy:
        learner_stats:
          cur_kl_coeff: 0.26362734606122995
          cur_lr: 5.000000000000001e-05
          entropy: 1.5354156348440382
          entropy_coeff: 0.009999999999999998
          kl: 0.011451674264380587
          policy_loss: -0.04258376900106668
          total_loss: -0.054573836751903096
          vf_explained_var: -0.7680838704109192
          vf_loss: 0.000345116782101387
    num_agent_steps_sampled: 643000
    num_agent_steps_trained: 643000
    num_steps_sampled: 643000
    num_steps_trained: 643000
  iterations_since_restore: 643
  node_ip: 10.55.229.8

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_my_env_d74d3_00000,RUNNING,10.55.229.87:333,643,13915.4,643000,0,0,0,995.57
